In [2]:
import numpy as np
from Data_Savior_J import load_file

Xz = load_file("./classifier_data/walk1.data")
Xz = np.vstack((Xz,load_file("./classifier_data/walk1U.data")))
Xz = np.vstack((Xz,load_file("./classifier_data/walk1D.data")))

Xz = np.vstack((Xz,load_file("./classifier_data/walk2.data")))
Xz = np.vstack((Xz,load_file("./classifier_data/walk2U.data")))
Xz = np.vstack((Xz,load_file("./classifier_data/walk2D.data")))

Xz = np.vstack((Xz,load_file("./classifier_data/walk3.data")))
Xz = np.vstack((Xz,load_file("./classifier_data/walk3U.data")))
Xz = np.vstack((Xz,load_file("./classifier_data/walk3D.data")))

Xz = np.vstack((Xz,load_file("./classifier_data/walk4.data")))
Xz = np.vstack((Xz,load_file("./classifier_data/walk4U.data")))
Xz = np.vstack((Xz,load_file("./classifier_data/walk4D.data")))

Xz = np.vstack((Xz,load_file("./classifier_data/walk5.data")))
Xz = np.vstack((Xz,load_file("./classifier_data/walk5U.data")))
Xz = np.vstack((Xz,load_file("./classifier_data/walk5D.data")))

## Vetor de features para classificação:

$X_c = [a~| ~av~| ~aa~| ~l\_a~| ~l\_av~| ~l\_aa~| ~pos\_foot\_r~| ~pos\_foot\_l~| ~vz\_r~| ~vz\_l~| ~C]$
$X_c = [0~|~~1~~| ~~2~~| ~~~3~~| ~~~4~~~| ~~~~5~~| ~~~~~~~6~~~~~~~~~~| ~~~~~~~~7~~~~~~~~| ~~~8~~~| ~~~~9~~| 10]$

#### $a \rightarrow$ ângulo do joelho direito; $av \rightarrow$ velocidade angular do joelho direito; $aa \rightarrow$ aceleração angular do joelho direito;
#### $a \rightarrow$ ângulo do joelho esquerdo; $av \rightarrow$ velocidade angular do joelho esquerdo; $aa \rightarrow$ aceleração angular do joelho esquerdo; 
#### $pos\_foot\_r \rightarrow$ posição do tornozelo direito em relação ao sacro; $pos\_foot\_l \rightarrow$ posição do tornozelo esquerdo em relação ao sacro;
#### $vz\_r \rightarrow$ velocidade do trocanter direito no eixo z; $vz\_l \rightarrow$ velocidade do trocanter esquerdo no eixo z;
#### $C \rightarrow$ indice de classificação
## Indice de classificação $"c"$:
#### $C = 0 \rightarrow$ Marcha normal;
#### $C = 1 \rightarrow$ Marcha de subida de escada;
#### $C = 2 \rightarrow$ Marvha de descidade escada.

In [19]:
import numpy as np

X = Xz[:,[0,1,2,3,4,5,6,7,8,9]]

yz = Xz[:,[10]]
y = np.array([])
for i in range(len(yz)):
    y = np.hstack((y,yz[i]))

X.shape, y.shape

((817, 10), (817,))

In [20]:
np.unique(y) # possíveis valores de y

array([ 0.,  1.,  2.])

In [21]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, 
                                                    random_state=10)

In [22]:

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)


In [23]:
print X_train_std.shape

(612, 10)


# SVM RBF/Polynomial Classifier

In [24]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, datasets

print 'Processing time RBF'
%time rbf_svc = svm.SVC(kernel='rbf', gamma=0.2, C=6, decision_function_shape='ovr').fit(X_train_std, y_train)
print ''
print 'Processing time Polynomial'
%time poly_svc = svm.SVC(kernel='poly', degree=2, coef0=4.7, C=48.9, decision_function_shape='ovr').fit(X_train_std, y_train)

def run_svm(svc, X_test_std, y_test):
    y_pred = svc.predict(X_test_std)
    from sklearn.metrics import accuracy_score
    if (svc==rbf_svc):
        print ('SVM-RBF accuracy:---------->%.2f %%' % (accuracy_score(y_test, y_pred)*100))
    elif(svc==poly_svc):
        print ('SVM-Polynomial accuracy:--->%.2f %%' % (accuracy_score(y_test, y_pred)*100))
        
run_svm(rbf_svc, X_test_std, y_test)
run_svm(poly_svc, X_test_std, y_test)

Processing time RBF
CPU times: user 12 ms, sys: 0 ns, total: 12 ms
Wall time: 10.2 ms

Processing time Polynomial
CPU times: user 12 ms, sys: 0 ns, total: 12 ms
Wall time: 12.8 ms
SVM-RBF accuracy:---------->99.02 %
SVM-Polynomial accuracy:--->99.02 %


# Gaussian Naive Bayes Classifier

In [25]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()
gnb.fit(X_train_std, y_train)

pred = gnb.predict(X_test_std)

from sklearn.metrics import accuracy_score
print ('ClassifyGNB accuracy:---------->%.2f %%' % (accuracy_score(y_test, pred)*100))

ClassifyGNB accuracy:---------->87.32 %


# Random Forest Classifier

In [49]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=100)

print 'Training time'
%time rfc = rfc.fit(X_train_std, y_train)

print''

print 'Predict time'
%time pred = rfc.predict(X_test_std)

print''

from sklearn.metrics import accuracy_score
print ('ClassifyRF accuracy:---------->%.2f %%' % (accuracy_score(y_test, pred)*100))

Training time
CPU times: user 232 ms, sys: 0 ns, total: 232 ms
Wall time: 231 ms

Predict time
CPU times: user 44 ms, sys: 0 ns, total: 44 ms
Wall time: 42.9 ms

ClassifyRF accuracy:---------->99.51 %


# Decision Tree Classifier

In [50]:
from sklearn import tree
dtc = tree.DecisionTreeClassifier(min_samples_split = 20)
dtc = dtc.fit(X_train_std, y_train)

print''

print 'Predict time'
%time pred = dtc.predict(X_test_std)

print''

from sklearn.metrics import accuracy_score
print ('ClassifyDT accuracy:---------->%.2f %%' % (accuracy_score(y_test, pred)*100))


Predict time
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 485 µs

ClassifyDT accuracy:---------->95.12 %


# Ada Boost Classifier

In [51]:
from sklearn.ensemble import AdaBoostClassifier
abc = AdaBoostClassifier(n_estimators=100, learning_rate = 2.5395)
abc = abc.fit(X_train_std, y_train)
print''

print 'Predict time'
%time pred = abc.predict(X_test_std)

print''

from sklearn.metrics import accuracy_score


print ('ClassifyAB accuracy:---------->%.2f %%' % (accuracy_score(y_test, pred)*100))


Predict time
CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 8.27 ms

ClassifyAB accuracy:---------->83.90 %


# KNeighbors Classifier

In [52]:
from sklearn.neighbors import KNeighborsClassifier
knc = KNeighborsClassifier(n_neighbors=2, weights = 'distance', algorithm = 'auto', p = 1)
knc = knc.fit(X_train_std, y_train)
print''

print 'Predict time'
%time pred = knc.predict(X_test_std)

print''

from sklearn.metrics import accuracy_score



print ('ClassifyKN accuracy:---------->%.2f %%' % (accuracy_score(y_test, pred)*100))



Predict time
CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 2.58 ms

ClassifyKN accuracy:---------->99.02 %


# Classificadores Utilizados:

### knc : KNeighborsClassifier; 
### abc : AdaBoostClassifier; 
### dtc : tree.DecisionTreeClassifier; 
### rfc : RandomForestClassifier; 
### gnb : GaussianNB; 
### rbf_svc : SVMClassifierRBF; 
### poly_svc = SVMClassifierPoly

In [53]:
from sklearn.ensemble import VotingClassifier
eclf1 = VotingClassifier(estimators=[('knc', knc), ('abc', abc), ('dtc', dtc), ('rfc', rfc), ('gnb', gnb), 
                                     ("rbf_svc", rbf_svc), ('poly_svc', poly_svc)], voting='hard',weights=[3,1,1,7,1,4,3])
eclf1 = eclf1.fit(X_train_std, y_train)

In [54]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import VotingClassifier

for clf, label in zip([knc, abc, dtc, rfc, gnb, rbf_svc, poly_svc, eclf1], ['KNieghbors', 'Ada Boost', 'Decision Tree', 
                                                                           'Random Forest', 'Naive Bayes', 'SVM-RBF', 
                                                                           'SVM-Poly', 'Ensemble']):
    scores = (cross_val_score(clf, X_train_std, y_train, cv=5, scoring='accuracy'))*100
    #print("Accuracy: %0.2f %%(+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))
    print("Accuracy: %0.2f %% [%s]" % (scores.mean(), label))

Accuracy: 96.73 % [KNieghbors]
Accuracy: 73.82 % [Ada Boost]
Accuracy: 91.82 % [Decision Tree]
Accuracy: 99.18 % [Random Forest]
Accuracy: 79.09 % [Naive Bayes]
Accuracy: 96.89 % [SVM-RBF]
Accuracy: 95.92 % [SVM-Poly]
Accuracy: 99.67 % [Ensemble]


In [55]:


import sklearn

print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 0.18.


In [57]:
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import CCA
from sklearn import cross_validation

pca = PCA(n_components=10)# adjust yourself
pca.fit(X_train)
X_t_train = pca.transform(X_train)
X_t_test = pca.transform(X_test)

clf = rfc
clf.fit(X_t_train, y_train)
print 'score', clf.score(X_t_test, y_test)

clf.predict(X_t_test)

scores = (cross_val_score(clf, X_train_std, y_train, cv=5, scoring='accuracy'))*100
print("Accuracy: %0.2f %%" % (scores.mean()))

score 0.985365853659
Accuracy: 99.34 %
